In [3]:
import time
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")

url = 'https://www.nguyenkim.com/laptop-may-tinh-xach-tay'
driver.get(url)

def get_product_links(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    product_grid = soup.find('div', class_ = 'nk-product-cate-style-grid nk-product-collection nk-product-laptopmaytinhxachtay clearfix')
    links = product_grid.select('a.product-render')
    product_links = [link['href'] for link in links] if links else None
    return product_links

all_product_links = []
# Lặp qua từng trang
for page_number in range(1,7): 
    # Truy cập trang
    driver.get(url + f'/page-{page_number}/')

    # Chờ để đảm bảo trang đã được tải đủ
    time.sleep(2)

    # Lấy dữ liệu từ trang hiện tại
    current_page_source = driver.page_source
    current_product_links = get_product_links(current_page_source)

    # In ra các link sản phẩm từ trang hiện tại
    for link in current_product_links:
        all_product_links.append(link)
print(all_product_links)
# Đóng trình duyệt khi hoàn thành
driver.quit()

['https://www.nguyenkim.com/laptop-dell-inspiron-14-5420-i5-1235u-dgdcg1.html', 'https://www.nguyenkim.com/laptop-msi-modern-14-r7-7730u-c7m-221vn.html', 'https://www.nguyenkim.com/laptop-acer-aspire-5-a514-55-5954-i5-1235u-nx-k5bsv-001.html', 'https://www.nguyenkim.com/laptop-acer-aspire-3-a314-23m-r4tx-r5-7520u-nx-kexsv-001.html', 'https://www.nguyenkim.com/laptop-hp-240-g9-i5-1235u-6l1y2pa.html', 'https://www.nguyenkim.com/laptop-asus-vivobook-15-i5-1335u-x1504va-nj070w.html', 'https://www.nguyenkim.com/laptop-dell-vostro-15-3530-i5-1335u-80gg9.html', 'https://www.nguyenkim.com/laptop-asus-tuf-gaming-f15-i5-11400h-fx506hf-hn014w.html', 'https://www.nguyenkim.com/laptop-dell-vostro-3430-i5-1335u-71011900.html', 'https://www.nguyenkim.com/laptop-dell-inspiron-15-3530-i7-1355u-71011775.html', 'https://www.nguyenkim.com/laptop-dell-inspiron-3530-i5-1335u-n5i5791w1.html', 'https://www.nguyenkim.com/laptop-asus-vivobook-14-oled-i5-13500h-a1405va-km095w.html', 'https://www.nguyenkim.com/la

In [4]:
print(len(all_product_links))

179


In [5]:

def get_product_info(all_product_links):
    driver.get(all_product_links)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    try:
        model_element = soup.find('td', class_='title', string='Model:')
        # Lấy giá trị model từ thẻ <td> tiếp theo
        if model_element:
            product_id = model_element.find_next_sibling('td', class_='value').text.strip()
        else:
            product_id = ''
    except:
        product_id = ''
    try:
        product_name = soup.find('h1', class_ = 'product_info_name').text.strip()
    except:
        product_name = ''
    try:
        product_url = soup.find('link', {'rel': 'canonical'}).get('href')
    except:
        product_url = ''
    try:
        discounted_price = soup.find('span', class_ = 'nk-price-final').text.strip()
    except:
        discounted_price = ''
    try:
        original_price = soup.find('div', class_ = 'product_info_price_value-real').text.strip()
    except:    
        original_price = discounted_price
    try:
        discounted_percent = soup.find('span', class_ = 'nk-product-discount-percent').text.strip().replace('-', '')
    except:
        discounted_percent = ''
    try:
        installment_payment = soup.find('span', class_ = 'nk-sticker nk-installment').text.strip()
    except:
        installment_payment = ''

    # Khuyến mại
    max_promotion = 8
    promotions = []
    try:
        promo_items = soup.find('div', class_ = 'title 222').text.strip()
        # Tạo một từ điển để lưu các giá trị
        
        if promo_items:
            variable_name = f"Promotion"
            promotions.append((variable_name, promo_items))
    except:
        pass
    try:
        promo_items_more = soup.find_all('span', class_ = 'promo_text') 
        if promo_items_more:
        # Lấy danh sách các thông tin khuyến mãi
            for i, t in enumerate(promo_items_more, start=1):
                variable_name = f"Promotion__more{i}"
                promotion_text = t.text.strip().replace('\xa0', ' ')
                if promotion_text != '':
                    promotions.append((variable_name, promotion_text))
    except:
        pass
            
    try:
        promo_paragraphs = soup.select('.control-group.product-list-field p:not(.product_option_item)')
        if promo_paragraphs:
        # Lấy danh sách các thông tin khuyến mãi
            for i, t in enumerate(promo_paragraphs, start=1):
                variable_name = f"Promotion__more_2{i}"
                promotion_text_2 = t.text.strip()
                if promotion_text_2 != '':
                    promotions.append((variable_name, promotion_text_2))
    except:
        pass
    
    product_price = {
            'product_id': product_id,
            'product_name': product_name,
            'discounted_price': discounted_price,
            'original_price': original_price,
            'discounted_percent': discounted_percent,
            'installment_payment' : installment_payment,
            'shop_brand': 'Nguyễn Kim'
        }
    # Thêm các biến khuyến mại vào dictionary
    for i in range(1, max_promotion + 1):
        key = f'promotion_{i}'
        # Nếu i vượt quá số lượng thực sự của khuyến mại, thiết lập giá trị là None
        if i <= len(promotions):
                value = promotions[i-1][1]
        else:
            value = ''
        product_price[key] = value

    product_price['product_url'] = product_url

    return product_price

driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
# Duyệt qua các đường link sản phẩm và lấy thông tin
data = []
for product_link in all_product_links:
    product_price = get_product_info(product_link)
    data.append(product_price)
    print(product_price)

driver.quit()

{'product_id': 'INSPIRON5420DGDCG1', 'product_name': 'Laptop Dell Inspiron 14 5420 i5-1235U/16GB/512GB/Win11 (DGDCG1)', 'discounted_price': '19.990.000đ', 'original_price': '22.990.000đ', 'discounted_percent': '13%', 'installment_payment': 'Trả góp 0%', 'shop_brand': 'Nguyễn Kim', 'promotion_1': 'Tặng túi hoặc ba lô đựng Laptop', 'promotion_2': 'Giảm 500.000đ khi thanh toán bằng thẻ tín dụng Sacombank', 'promotion_3': 'Giảm 500.000đ khi thanh toán bằng thẻ tín dụng UOB', 'promotion_4': 'Nhập mã NKTET giảm 1% tối đa 300,000đ khi thanh toán qua VNPay (áp dụng tại cửa hàng Nguyễn Kim)', 'promotion_5': '', 'promotion_6': '', 'promotion_7': '', 'promotion_8': '', 'product_url': 'https://www.nguyenkim.com/laptop-dell-inspiron-14-5420-i5-1235u-dgdcg1.html'}
{'product_id': 'MODERN14C7M-221VN', 'product_name': 'Laptop MSI Modern 14 R7-7730U/8GB/512GB/Win11 (C7M-221VN)', 'discounted_price': '11.990.000đ', 'original_price': '14.490.000đ', 'discounted_percent': '17%', 'installment_payment': 'Trả g

In [6]:
import pandas as pd
df = pd.DataFrame(data)
df

,product_id,product_name,discounted_price,original_price,discounted_percent,installment_payment,shop_brand,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,promotion_6,promotion_7,promotion_8,product_url
0,INSPIRON5420DGDCG1,Laptop Dell Inspiron 14 5420 i5-1235U/16GB/512...,19.990.000đ,22.990.000đ,13%,Trả góp 0%,Nguyễn Kim,Tặng túi hoặc ba lô đựng Laptop,Giảm 500.000đ khi thanh toán bằng thẻ tín dụng...,Giảm 500.000đ khi thanh toán bằng thẻ tín dụng...,"Nhập mã NKTET giảm 1% tối đa 300,000đ khi than...",,,,,https://www.nguyenkim.com/laptop-dell-inspiron...
1,MODERN14C7M-221VN,Laptop MSI Modern 14 R7-7730U/8GB/512GB/Win11 ...,11.990.000đ,14.490.000đ,17%,Trả góp 0%,Nguyễn Kim,Giảm 500.000đ khi thanh toán bằng thẻ tín dụng...,Giảm 500.000đ khi thanh toán bằng thẻ tín dụng...,"Nhập mã NKTET giảm 1% tối đa 300,000đ khi than...",,,,,,https://www.nguyenkim.com/laptop-msi-modern-14...
2,A514-55-5954,Laptop Acer Aspire 5 A514-55-5954 i5-1235U/8GB...,14.990.000đ,19.990.000đ,25%,Trả góp 0%,Nguyễn Kim,Tặng túi hoặc ba lô đựng Laptop,Giảm 500.000đ khi thanh toán bằng thẻ tín dụng...,Giảm 500.000đ khi thanh toán bằng thẻ tín dụng...,"Nhập mã NKTET giảm 1% tối đa 300,000đ khi than...",,,,,https://www.nguyenkim.com/laptop-acer-aspire-5...
3,A314-23M-R4TX,Laptop Acer Aspire 3 A314-23M-R4TX R5-7520U/8G...,11.990.000đ,13.290.000đ,10%,Trả góp 0%,Nguyễn Kim,Tặng túi hoặc ba lô đựng Laptop,Giảm 500.000đ khi thanh toán bằng thẻ tín dụng...,Giảm 500.000đ khi thanh toán bằng thẻ tín dụng...,"Nhập mã NKTET giảm 1% tối đa 300,000đ khi than...",,,,,https://www.nguyenkim.com/laptop-acer-aspire-3...
4,6L1Y2PA,Laptop HP 240 G9 i5-1235U/8GB/512GB/Win11 6L1Y2PA,14.690.000đ,18.790.000đ,22%,Trả góp 0%,Nguyễn Kim,Tặng túi hoặc ba lô đựng Laptop,Giảm 500.000đ khi thanh toán bằng thẻ tín dụng...,Giảm 500.000đ khi thanh toán bằng thẻ tín dụng...,"Nhập mã NKTET giảm 1% tối đa 300,000đ khi than...",,,,,https://www.nguyenkim.com/laptop-hp-240-g9-i5-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,S530UA-BQ145T,LAPTOP ASUS S530UA-BQ145T,13.690.000đ,13.690.000đ,,Trả góp 0%,Nguyễn Kim,,,,,,,,,https://www.nguyenkim.com/laptop-asus-s530ua-b...
175,S430FA-EB459T,Laptop Asus S430FA-EB459T,14.490.000đ,14.490.000đ,,Trả góp 0%,Nguyễn Kim,,,,,,,,,https://www.nguyenkim.com/laptop-asus-s430fa-e...
176,S430FA-EB253T,LAPTOP ASUS S430FA-EB253T,18.990.000đ,18.990.000đ,,Trả góp 0%,Nguyễn Kim,,,,,,,,,https://www.nguyenkim.com/laptop-asus-s430fa-e...
177,8QN33PA,HP Notebook 14S-DQ1020TU i5-10350G1 14 inch 8Q...,14.690.000đ,14.690.000đ,,Trả góp 0%,Nguyễn Kim,,,,,,,,,https://www.nguyenkim.com/hp-notebook-14s-dq10...


In [7]:
df.to_excel('nguyenkim_price.xlsx', index=False)